In [197]:
import pandas as pd
import numpy as np
import sqlite3

In [198]:
def func_load_dataset(file_path, table_name = "bikedata_tb"):
    
    # file_path >> .db file path
    # table_name >> table name for the .db file
    
    # establish connection with the .db file
    con = sqlite3.connect(str(file_path))
    
    # Modify SQL query to get the required columns.
    # columns fetched here are bike_ids, lon, lat, battery_level, my_time_stamp
    
    df = pd.read_sql_query("select bike_id, lon, lat, battery_level, my_time_stamp from "+table_name, con)
    
    # IMPORTANT to close connection with the existing .db file after processing.
    con.close()
    return df

In [199]:
class GetIntersectingPoints:

    def __init__(self, lon, lat):
        self.lon = lon
        self.lat = lat

        self.lat_corner1 = 38.995418
        self.lon_corner1 = -77.041078

        self.lat_corner2 = 38.892845
        self.lon_corner2 = -76.909496

        self.lat_corner3 = 38.828800
        self.lon_corner3 = -76.9992657

        self.lat_corner4 = 38.934471
        self.lon_corner4 = -77.119045

    def get_int_points_on_line12(self):

        m12 = (self.lon_corner2 - self.lon_corner1) / (self.lat_corner2 - self.lat_corner1)
        m14 = (self.lon_corner4 - self.lon_corner1) / (self.lat_corner4 - self.lat_corner1)

        intersect_lon_12 = self.lon + (m14 / (m12 - m14))*(m12*(self.lat_corner1 - self.lat) + (self.lon - self.lon_corner1))
        intersect_lat_12 = (m12*self.lat_corner1 - m14*self.lat + self.lon - self.lon_corner1) / (m12 - m14)

        return intersect_lon_12, intersect_lat_12

    def get_int_points_on_line14(self):

        m12 = (self.lon_corner2 - self.lon_corner1)/(self.lat_corner2 - self.lat_corner1)
        m14 = (self.lon_corner4 - self.lon_corner1)/(self.lat_corner4 - self.lat_corner1)

        intersect_lon_14 = self.lon_corner1 + (m14/(m12 - m14))*(m12*(self.lat - self.lat_corner1) + (self.lon_corner1 - self.lon))
        intersect_lat_14 = (m12*self.lat - m14*self.lat_corner1 + self.lon_corner1 - self.lon)/(m12 - m14)

        return intersect_lon_14, intersect_lat_14

    

In [200]:
class ComputeDistance:

    def __init__(self, lon0, lat0, lon1, lat1):
        self.lon0 = lon0
        self.lat0 = lat0

        self.lon1 = lon1
        self.lat1 = lat1

    def calc_distance(self):

        lon0 = np.radians(np.float64(self.lon0))
        lat0 = np.radians(np.float64(self.lat0))

        lon1 = np.radians(np.float64(self.lon1))
        lat1 = np.radians(np.float64(self.lat1))

        value = np.sin(lat0) * np.sin(lat1) + (np.cos(lat0) * np.cos(lat1) * np.cos(lon0 - lon1))
        dist = 6371.01 * 1000 * np.arccos(value)

        return dist

In [201]:
class Get12Params:
    def __init__(self, lon, lat, steps12):

        self.lon_corner1 = -77.041078
        self.lat_corner1 = 38.995418

        self.lon_corner2 = -76.909496
        self.lat_corner2 = 38.892845

        self.lat = lat
        self.lon = lon
        self.steps12 = steps12

    def calc_params(self):

        int_lon_12, int_lat_12 = GetIntersectingPoints(self.lon, self.lat).get_int_points_on_line12()

        dist12 = ComputeDistance(self.lon_corner1, self.lat_corner1, self.lon_corner2, self.lat_corner2).calc_distance()
        dist1int = ComputeDistance(self.lon_corner1, self.lat_corner1, int_lon_12, int_lat_12).calc_distance()
        

        param12 = int(np.floor((dist1int*self.steps12)/dist12))

        return param12

In [202]:
class Get14Params:
    def __init__(self, lon, lat, steps14):

        self.lon_corner1 = -77.041078
        self.lat_corner1 = 38.995418

        self.lat_corner4 = 38.934471
        self.lon_corner4 = -77.119045

        self.lat = lat
        self.lon = lon
        self.steps14 = steps14

    def calc_params(self):

        int_lon_14, int_lat_14 = GetIntersectingPoints(self.lon, self.lat).get_int_points_on_line14()
        
        dist14 = ComputeDistance(self.lon_corner1, self.lat_corner1, self.lon_corner4, self.lat_corner4).calc_distance()
        dist1int = ComputeDistance(self.lon_corner1, self.lat_corner1, int_lon_14, int_lat_14).calc_distance()

        param14 = int(np.floor((dist1int*self.steps14)/dist14))

        return param14

In [203]:
import os
df = func_load_dataset("bird_scooters_05_02_2020_01_09_45.db")
df['lon'] = df['lon'].astype(np.float64)
df['lat'] = df['lat'].astype(np.float64)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238082 entries, 0 to 238081
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   bike_id        238082 non-null  object 
 1   lon            238082 non-null  float64
 2   lat            238082 non-null  float64
 3   battery_level  238082 non-null  object 
 4   my_time_stamp  238082 non-null  object 
dtypes: float64(2), object(3)
memory usage: 9.1+ MB


In [205]:
df_sq = pd.read_csv("washington_grid_square_grid_side_size_100m_updated.csv")
df_sq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15840 entries, 0 to 15839
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  15840 non-null  int64  
 1   square_id   15840 non-null  int64  
 2   x           15840 non-null  int64  
 3   y           15840 non-null  int64  
 4   lon_cor1    15840 non-null  float64
 5   lat_cor1    15840 non-null  float64
 6   lon_cor2    15840 non-null  float64
 7   lat_cor2    15840 non-null  float64
 8   lon_cor3    15840 non-null  float64
 9   lat_cor3    15840 non-null  float64
 10  lon_cor4    15840 non-null  float64
 11  lat_cor4    15840 non-null  float64
 12  x_max       15840 non-null  int64  
 13  y_max       15840 non-null  int64  
 14  s12         15840 non-null  int64  
 15  s14         15840 non-null  int64  
dtypes: float64(8), int64(8)
memory usage: 1.9 MB


In [206]:
s12 = df_sq['s12'][0]
s14 = df_sq['s14'][0]
print("steps")
print("s12 ", s12, "s14 ", s14)

In [208]:
class assign_gbfs:

  def __init__(self,df_gbfs, s12, s14):
    self.df_gbfs = df_gbfs
    self.s12 = s12
    self.s14 = s14

  def add_params(self):
    temp_df = self.df_gbfs
    temp_df['param_12'] = temp_df.apply(lambda x: Get12Params(x.lon, x.lat, self.s12).calc_params(), axis = 1)
    temp_df['param_14'] = temp_df.apply(lambda x: Get14Params(x.lon, x.lat, self.s14).calc_params(), axis = 1)
    return temp_df

In [209]:
df = assign_gbfs(df_gbfs=df, s12 = df_sq['s12'][0], s14 = df_sq['s14'][0]).add_params()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238082 entries, 0 to 238081
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   bike_id        238082 non-null  object 
 1   lon            238082 non-null  float64
 2   lat            238082 non-null  float64
 3   battery_level  238082 non-null  object 
 4   my_time_stamp  238082 non-null  object 
 5   param_12       238082 non-null  int64  
 6   param_14       238082 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 12.7+ MB
